In [1]:
import tensorflow as tf
import numpy as np
from numpy import linalg as LA
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, shape=[784, None])
y_true = tf.placeholder(tf.float32, shape=[10, None])
W = tf.Variable(tf.zeros([10,784]))
# b = tf.Variable(tf.zeros([10,1]))
y_pred = tf.matmul(W, x) 
y_pred.get_shape()

TensorShape([Dimension(10), Dimension(None)])

In [63]:
def tf_frobenius_norm(M):
    return tf.reduce_sum(M ** 2) ** 0.5

# to implement nuclear norm
def tf_nuclear_norm(M):
    st, ut, vt = tf.svd(M,  full_matrices = False)
    st2 = tf.diag(st)
    st_r = tf.matmul(ut, tf.matmul(st2, tf.transpose(vt)))
    print('vish', ut.shape, st2.shape, tf.transpose(vt).shape, st_r.shape)
    
    uk = tf.reshape(ut[:, 0], [10, 1])
    vk = tf.reshape(vt[:, 0], [1, 784])
    sk = tf.matmul(uk, vk)
    sk = st[0] * sk
    print(st.shape, ut.shape, vt.shape)
    print('before', type(sk), sk.shape)
    return sk, st[0], st_r, M

# def tf_nuclear_norm_pm(M):
#     un = np.zeros([10, 1])
#     un[0][0] = 1
#     u = tf.constant(un)
#     vn = np.zeros([1, 784])
#     vn[0][0] = 1
#     v = tf.constant(vn)
#     for _ in range(20): 
#         u = tf.matmul(M, x)
#         v = tf.

def Sgdnm(grad, wt):
    return (grad / tf_frobenius_norm(grad))

def Cgd_Fn(grad, wt):
    return ((1 - alpha ) / alpha) * (wt + lam1 * grad / tf_frobenius_norm(grad))

def Cgd_Nn(grad, wt):
    nn, st, st_r, M = tf_nuclear_norm(grad)
    return ((1 - alpha ) / alpha) * (wt - lam2 * nn), st, st_r, M


In [64]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# hyper-parameters
# alpha = tf.placeholder_with_default(tf.constant(0.5), tf.constant(0.001).shape)
global_step = tf.Variable(0, trainable=False)
start_train = 0.9999 # Requires very high lambda for Cgd_Fn
# k=1, start_train = 1, decay_rate = 1 ---> 1/t learning rate
k = 1
alpha = tf.train.inverse_time_decay(start_train, global_step, k, 0)
lam1 = tf.placeholder_with_default(tf.constant(4.0), tf.constant(10000.0).shape)
lam2 = tf.placeholder_with_default(tf.constant(4.0), tf.constant(100.0).shape)

# Gradient Descent optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate = alpha)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss)

# SGD update
gv_sgd = [(gv[0], gv[1]) for gv in grads_and_vars]
optimizer_gv_sgd = opt.apply_gradients(gv_sgd, global_step=global_step)
g0_sgd = grads_and_vars[0][0]
w0_sgd = grads_and_vars[0][1]
s0_sgd = w0_sgd
w1_sgd = w0_sgd - alpha * s0_sgd


# Normalized SGD update
gv_nsgd = [(Sgdnm(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_nsgd = opt.apply_gradients(gv_nsgd, global_step=global_step)
g0_nsgd = grads_and_vars[0][0]
w0_nsgd = grads_and_vars[0][1]
s0_nsgd = Sgdnm(g0_nsgd, w0_nsgd)
w1_nsgd = w0_nsgd - alpha * s0_nsgd


# CGD with FN
gv_cgd_fn = [(Cgd_Fn(gv[0], gv[1]), gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_fn = opt.apply_gradients(gv_cgd_fn, global_step=global_step)
g0_cgd_fn = grads_and_vars[0][0]
w0_cgd_fn = grads_and_vars[0][1]
s0_cgd_fn = Cgd_Fn(g0_cgd_fn, w0_cgd_fn)
w1_cgd_fn = w0_cgd_fn - alpha * s0_cgd_fn

# CGD with NN
g0_cgd_nn = grads_and_vars[0][0]
w0_cgd_nn = grads_and_vars[0][1]
s0_cgd_nn, st, st_r, M = Cgd_Nn(g0_cgd_nn, w0_cgd_nn)
gv_cgd_nn = [(s0_cgd_nn, gv[1]) for gv in grads_and_vars]
optimizer_gv_cgd_nn = opt.apply_gradients(gv_cgd_nn, global_step=global_step)
w1_cgd_nn = w0_cgd_nn - alpha * s0_cgd_nn

correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.argmax(y_true, 0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

vish (10, 10) (10, 10) (10, 784) (10, 784)
(10,) (10, 10) (784, 10)
before <class 'tensorflow.python.framework.ops.Tensor'> (10, 784)


# Normalized gradient descent

In [65]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(1000):
#         batch = mnist.train.next_batch(100)
#         feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
#         train_accuracy, loss_, gv_, W_ = sess.run([accuracy, loss, gv_nsgd, W], feed_dict)
#         w1_nsgd_, w0_nsgd_, s0_nsgd_= sess.run([w1_nsgd, w0_nsgd, s0_nsgd], feed_dict)
#         alpha_ = sess.run([alpha], feed_dict)
            
#         if i % 1 == 0:
#             print('train_accuracy=', train_accuracy, 'loss=', loss_)
#             print('NSgd iterates: w(t+1) =', LA.norm(w1_nsgd_), 'w(t) =', LA.norm(w0_nsgd_), 's(t) =', LA.norm(s0_nsgd_))
#             print('alpha', alpha_)
#         sess.run(optimizer_gv_nsgd, feed_dict)
    
#     feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
#     test_accuracy = sess.run(accuracy, feed_dict)
#     print('test_accuracy=', test_accuracy)

# Frobenius norm

In [66]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(1000):
#         batch = mnist.train.next_batch(100)
#         batch1 = batch[:]
#         feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
#         train_accuracy, loss_, gv_, W1_ = sess.run([accuracy, loss, gv_cgd_fn, W], feed_dict)
#         w1_cgd_fn_, w0_cgd_fn_, s0_cgd_fn_ = sess.run([w1_cgd_fn, w0_cgd_fn, s0_cgd_fn], feed_dict)
#         alpha_ = sess.run([alpha], feed_dict)
            
#         if i % 100 == 0:
#             print('train_accuracy=', train_accuracy, 'loss value =',loss_)
#             print('frob_nrom of iterates: w(t+1) =', LA.norm(w1_cgd_fn_), 'w(t) =', LA.norm(w0_cgd_fn_), 's(t) =', LA.norm(s0_cgd_fn_))
#             print('alpha', alpha_)
#         sess.run(optimizer_gv_cgd_fn, feed_dict)
    
#     feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
#     test_accuracy = sess.run(accuracy, feed_dict)
#     print('test_accuracy', test_accuracy)

# Nuclear Norm

In [68]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        batch = mnist.train.next_batch(100)
        batch2 = batch[:]
        feed_dict = {x: np.transpose(batch[0]), y_true: np.transpose(batch[1])}
        train_accuracy, loss_, gv_, W2_ = sess.run([accuracy, loss, gv_cgd_nn, W], feed_dict)
        w1_cgd_nn_, w0_cgd_nn_, s0_cgd_nn_, st_, st_r_, M_ = sess.run([w1_cgd_nn, w0_cgd_nn, s0_cgd_nn, st, st_r, M], feed_dict)
        alpha_ = sess.run([alpha], feed_dict)
            
        if i % 1 == 0:
            print('train_accuracy=', train_accuracy, 'loss value =',loss_)
            print('nuclear_norm of iterates: w(t+1) =', LA.norm(w1_cgd_nn_), 'w(t) =', LA.norm(w0_cgd_nn_), 's(t) =', LA.norm(s0_cgd_nn_))
            print('alpha', alpha_)
        sess.run(optimizer_gv_cgd_nn, feed_dict)
    
    feed_dict={x: np.transpose(mnist.test.images), y_true: np.transpose(mnist.test.labels)}
    test_accuracy = sess.run(accuracy, feed_dict)
    print('test_accuracy', test_accuracy)

train_accuracy= 0.08 loss value = 46.0517
nuclear_norm of iterates: w(t+1) = 0.0065515 w(t) = 0.0 s(t) = 0.00655215
alpha [0.99989998]
train_accuracy= 0.09 loss value = 46.0517
nuclear_norm of iterates: w(t+1) = 0.0139243 w(t) = 0.0065515 s(t) = 0.00784853
alpha [0.99989998]
train_accuracy= 0.13 loss value = 46.0607
nuclear_norm of iterates: w(t+1) = 0.0203266 w(t) = 0.0139243 s(t) = 0.00657983
alpha [0.99989998]
train_accuracy= 0.09 loss value = 46.063
nuclear_norm of iterates: w(t+1) = 0.0265976 w(t) = 0.0203266 s(t) = 0.00652477
alpha [0.99989998]
train_accuracy= 0.09 loss value = 46.0657
nuclear_norm of iterates: w(t+1) = 0.0324852 w(t) = 0.0265976 s(t) = 0.00627808
alpha [0.99989998]
train_accuracy= 0.09 loss value = 46.071
nuclear_norm of iterates: w(t+1) = 0.0380327 w(t) = 0.0324852 s(t) = 0.00591359
alpha [0.99989998]
train_accuracy= 0.16 loss value = 46.07
nuclear_norm of iterates: w(t+1) = 0.0439815 w(t) = 0.0380327 s(t) = 0.00637539
alpha [0.99989998]
train_accuracy= 0.09 lo

In [61]:
LA.norm(M_)

23.973583

In [62]:
LA.norm(st_r_)

23.973591

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ..., 
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]], dtype=bool)